# Webscrapping

## Webscrapping basique avec unique package beautifulsoup

### Installation des packages

In [1]:
!pip install lxml

import pandas as pd
import requests
from bs4 import BeautifulSoup
from pprint import pprint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 44.9 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


##### Dans un premier temps, on chercher à récupérer des informations sur la communication verte de l'entreprise Pfizer. Pour cela, on test le webscrapping sur différents sites internet : CNN, Europe PMC et Facebook

### CNN 

In [2]:
nom_entreprise = "pfizer"
url = "https://edition.cnn.com/search?q={0}&from=0&size=10&page=1&sort=newest&types=all&section=".format(nom_entreprise)

response = requests.get(url)

if response.status_code == 200:
    # Utilisez response.text pour obtenir le contenu sous forme de chaîne de caractères
    html_content = response.text

    # Utilisez BeautifulSoup avec html_content
    soup = BeautifulSoup(html_content, 'html.parser')
        
else:
    print(f"La requête a échoué avec le code d'état {response.status_code}")

search_results_count_div = soup.find('div', class_='search__results-count')
search_results_count_div


<div class="search__results-count">
<strong id="search__query"></strong>
</div>

##### Conclusion : Aucun texte présent dans la balise recherchée...

### EUROPE PMC

In [3]:
from bs4 import BeautifulSoup
import requests

numero_article = 10677964
url = "https://europepmc.org/article/PMC/PMC{0}".format(numero_article)

response = requests.get(url)

if response.status_code == 200:
    # Utilisez response.text pour obtenir le contenu sous forme de chaîne de caractères
    html_content = response.text

    # Utilisez BeautifulSoup avec html_content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Récupérez la balise <h2> avec l'attribut id="article--free-full-text--header"
    h2_tag = soup.find('h2', {'id': 'article--free-full-text--header'})
    
    if h2_tag:
        print("Balise <h2> trouvée :", h2_tag.text)
    else:
        print("Balise <h2> non trouvée.")
    
else:
    print(f"La requête a échoué avec le code d'état {response.status_code}")


Balise <h2> non trouvée.


##### Conclusion : On ne trouve pas la basise qui permet d'afficher l'abstract de l'article

### Facebook

In [4]:
from bs4 import BeautifulSoup
import requests

url = "https://www.facebook.com/Pfizer/?locale=fr_FR"

response = requests.get(url)

if response.status_code == 200:
    # Utilisez response.text pour obtenir le contenu sous forme de chaîne de caractères
    html_content = response.text

    # Utilisez BeautifulSoup avec html_content
    soup = BeautifulSoup(html_content, 'html.parser')
    all_post = soup.find_all('div', class_='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z') 
    
else:
    print(f"La requête a échoué avec le code d'état {response.status_code}")


all_post


[]

##### Conclusion : On ne trouve pas les balises correspondant aux postes

##### Possible problème de la méthode utilisée précedemment : les sites internet recherchés sont dynamiques. Pour résoudre ce problème, on essaye le package selinium.

## Webscrapping avec selinium

### Installation des packages

In [5]:
!sudo apt-get update
!sudo apt install -y unzip xvfb libxi6 libgconf-2-4 -y
!sudo apt install chromium-chromedriver -y
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -O /tmp/chrome.deb
!sudo apt-get update
!sudo -E apt-get install -y /tmp/chrome.deb
!pip install chromedriver-autoinstaller selenium

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()


!pip install webdriver_manager

import selenium
from webdriver_manager.chrome import ChromeDriverManager

path_to_web_driver = ChromeDriverManager().install()


import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('--verbose') 


from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
service = Service(executable_path=path_to_web_driver)

browser = webdriver.Chrome(service=service,
                           options=chrome_options)

Get:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3631 B]
Get:2 https://dl.yarnpkg.com/debian stable InRelease [17.1 kB]
Get:3 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease [3960 B]
Get:4 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:6 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main amd64 Packages [252 kB]
Get:7 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main all Packages [2714 B]
Get:8 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable/main amd64 Packages [3650 B]
Get:9 https://dl.yarnpkg.com/debian stable/main amd64 Packages [11.1 kB]       
Get:10 https://dl.yarnpkg.com/debian stable/main all Packages [11.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]       
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease

### CNN 

In [6]:
nom_entreprise = "pfizer"
url = "https://edition.cnn.com/search?q={0}&from=0&size=10&page=1&sort=newest&types=all&section=".format(nom_entreprise)

browser.get(url)

# Obtenez le contenu de la page après que JavaScript ait pu mettre à jour le DOM
html_content = browser.page_source

# Utilisez BeautifulSoup comme précédemment
soup = BeautifulSoup(html_content, 'html.parser')

soup


<html><head></head><body></body></html>

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def scrape_cnn_for_pizer(start_year=2010, end_year=2023):
    # Initialiser le navigateur Selenium
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

 
    # Construire l'URL de recherche
    search_url = f"https://edition.cnn.com/search?q=Pfizer+environment"
    driver.get(search_url)

    time.sleep(5)  # Attendre que la page charge

    # Récupérer les titres des articles (ajuster le sélecteur en fonction de la structure de la page)
    articles = driver.find_elements(By.CSS_SELECTOR, 'h3.article-title')

    for article in articles:
        title = article.text
        print(f'{title}')

    time.sleep(2)  # Petite pause pour éviter de surcharger le serveur

    driver.quit()

scrape_cnn_for_pizer()


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /opt/google/chrome/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5624ad503f83 <unknown>
#1 0x5624ad1bccf7 <unknown>
#2 0x5624ad1f460e <unknown>
#3 0x5624ad1f126e <unknown>
#4 0x5624ad24180c <unknown>
#5 0x5624ad235e53 <unknown>
#6 0x5624ad1fddd4 <unknown>
#7 0x5624ad1ff1de <unknown>
#8 0x5624ad4c8531 <unknown>
#9 0x5624ad4cc455 <unknown>
#10 0x5624ad4b4f55 <unknown>
#11 0x5624ad4cd0ef <unknown>
#12 0x5624ad49899f <unknown>
#13 0x5624ad4f1008 <unknown>
#14 0x5624ad4f11d7 <unknown>
#15 0x5624ad503124 <unknown>
#16 0x7f08a0e0d609 start_thread


##### Conclusion : on fait pire qu'avec le package beautiful soup uniquement, on arrive pas à télécharger le code HTML

### Facebook

In [ ]:
url = "https://www.facebook.com/Pfizer/?locale=fr_FR"

browser.get(url)

# Obtenez le contenu de la page après que JavaScript ait pu mettre à jour le DOM
html_content = browser.page_source

# Utilisez BeautifulSoup comme précédemment
soup = BeautifulSoup(html_content, 'html.parser')
div_tags = soup.find_all('div', class_='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z')

div_tags

##### Conclusion : On arrive toujours pas à récupérer les postes

### Europe PMC

In [ ]:
numero_article = 10677964
url = "https://europepmc.org/article/PMC/PMC{0}".format(numero_article)

browser.get(url)

# Obtenez le contenu de la page après que JavaScript ait pu mettre à jour le DOM
html_content = browser.page_source

# Utilisez BeautifulSoup comme précédemment
soup = BeautifulSoup(html_content, 'html.parser')
h2_tag = soup.find('h2', {'id': 'article--free-full-text--header'})

if h2_tag:
    print("Balise <h2> trouvée :", h2_tag.text)
else:
    print("Balise <h2> non trouvée.")

##### Conclusion : pas d'amélioration